In [1]:
import requests
import re

from bs4 import BeautifulSoup as bs

import time
import random

import pandas as pd

import numpy as np

import signal

import os

import shutil

In [2]:
years = range(2018, 2024)

In [3]:
team_names=[
    '49ers',
    'Cardinals',
    'Falcons',
    'Ravens',
    'Buffalos',
    'Panthers',
    'Bears',
    'Bengals',
    'Browns',
    'Cowboys',
    'Broncos',
    'Lions',
    'Packers',
    'Texans',
    'Colts',
    'Jaguars',
    'Chiefs',
    'Raiders',
    'Chargers',
    'Rams',
    'Dolphins',
    'Vikings',
    'Patriots',
    'Saints',
    'Giants',
    'Jets',
    'Eagles',
    'Steelers',
    'Seahawks',
    'Buccaneers',
    'Titans',
    'Commanders'
]

In [4]:
team_names.sort()

In [5]:
def get_player_li (team: str, year: int):

    roster = pd.read_csv('NFL_Data_{}/{}/{}_roster'.format(year, team, team))

    roster = roster[:-1]

    #reset index
    roster.reset_index(drop=['index'], inplace=True)

    #get list of players
    players_li = list(roster['Player'].values)[:-1]
    
    return players_li

In [6]:
#loop through player_name list to get each player url
#url_number to differentiates similar players similar url names
#year to select which season

### this uses the first 2 characters of the first name
def player_url_func_1(player: str, url_number: int, year: int):
    
    '''
    player = str search for the player you're looking for
    url_number = int the number to distinguish between similar player names
    year = int which year you're searching in
    '''
    
    
    #split the names
    if player.isalpha() == False:
        
        #delete ' from names
        name_replace = player.replace("'", "")

        #leave out anything other than names
        split_name = name_replace.split()[:2]
            
            
    else:
        split_name = player.split()

    #players last name first character
    player_ln_1c = split_name[1][0]
    
    #players first two characters of first name
    player_fn_2c = split_name[0][:2]
        
    player_fn_2c.strip()
        
    #first four characters of last name
    if len(split_name[1]) < 4:
        player_ln_4c = split_name[1]+'x'
        
    else:
        player_ln_4c = split_name[1][:4]


    #the url
    #in order, the brackets to fill in
        #fist character from last name
        #first four characters from last name
        #first two characters of first name
        # number code for to differentiate similar url because it is based off names/initial
        #+1 when name doesn't match
        #year
    url = 'http://www.pro-football-reference.com/players/{}/{}{}{}/gamelog/{}/advanced/'

    #for the url code to differentiate names
    url_no = '{:02d}'.format(url_number)

    #url code
    url_requests = url.format(player_ln_1c, player_ln_4c, player_fn_2c, url_no, year)

    return url_requests

In [7]:
#loop through player_name list to get each player url
#url_number to differentiates similar players similar url names
#year to select which season

### this uses the first 3 characters of the first name
def player_url_func_2(player: str, url_number: int, year: int):
   
    '''
    Some url use a different number of characters from the player's name, to create up the url. 
    
    player = str search for the player you're looking for
    url_number = int the number to distinguish between similar player names
    year = int which year you're searching in
    '''
    
    #split the names
    if player.isalpha() == False:

        #delete ' from names
        name_replace = player.replace("'", "")

        #leave out anything other than names
        split_name = name_replace.split()[:2]

    else:
        split_name = player.split()

    #players last name first character
    player_ln_1c = split_name[1][0]

    #players first two characters of first name
    player_fn_3c = split_name[0][:3]


    #first four characters of last name
    if len(split_name[1]) < 4:
        player_ln_4c = split_name[1]+'xx'
    else:
        player_ln_4c = split_name[1][:5]


    #the url
    #in order, the brackets to fill in
        #fist character from last name
        #first four characters from last name
        #first two characters of first name
        # number code for to differentiate similar url because it is based off names/initial
            #+1 when name doesn't match
        #year
    url = 'http://www.pro-football-reference.com/players/{}/{}{}{}/gamelog/{}/advanced/'

    #for the url code to differentiate names
    url_no = '{:02d}'.format(url_number)

    #url code
    url_requests = url.format(player_ln_1c, player_ln_4c, player_fn_3c, url_no, year)

    return url_requests

In [8]:
def webscrape_player_name(url: str):
    '''
    Webscrape the for the player's name by generated url.
    
    url = str webscrapes url for player's name
    '''
    
    #request html
    html = requests.get(url).text

    time.sleep(random.randrange(5,7))

    #parse html
    soup = bs(html, 'html.parser')

    #find header for player name
    h1 = soup.find_all('h1')[0]


    try:
        #get player name
        span = h1.find_all('span')[0]

        name = span.contents[0]

    except:
        name = False


    return name

In [9]:
def get_player_url_1(search_player:str, year:int):
    
    '''
    Search for the correct player from the roster. 
    
    search_player = str Player you are searching for
    year = int The year of the roster youre using
    '''

    url_number_li = [0, 1, 2, 3, 4, 5, 6, 7, 20, 21, 44, 99, 'n/a']
    
    
    for url_number in url_number_li:
        try:
            
            #url for initial search for player name on url
            search_player_url = player_url_func_1(player=search_player, url_number=url_number, year=year)
            
            player1 = webscrape_player_name(url=search_player_url)
            
            # matcing which
            if player1 != search_player:
                pass
            
            else:
                break
        
        except:
            return 'n/a_' + search_player
        
    return search_player_url

In [10]:
def get_player_url_2(search_player:str, year:int):
    url_number= 1
    
    try:
        #url for initial search for player name on url
        search_player_url = player_url_func_2(player=search_player, url_number=url_number, year=year)
            
        #player scraped from url    
        player1 = webscrape_player_name(url=search_player_url)
            
            # matcing which
        if player1 != search_player:
            return 'n/a_' + search_player
        
    except:
        return 'n/a_' + search_player
        

    return search_player_url

In [11]:
# get final and correct url
##### maybe###########
def final_url(player: str, year: int):
    '''
    Connecting all the functions. 
    '''
    try:
        get_player_url = get_player_url_1(search_player=player, year=year)
        
        time.sleep(random.randrange(5,7))

        if get_player_url == 'n/a_'+player:
            get_player_url = get_player_url_2(search_player=player, year=year)

        
    except:
        return 'n/a_' + search_player

        
        
    return get_player_url

In [12]:
def webscrape_player_adv_log_table(player_url: str, team: str, year: int):
    '''
    url_li = teams roster url list in string
    '''
    try:
        #scrape table
        df_tables = pd.read_html(player_url)
        
        time.sleep(random.randrange(5,7))
        
        if len(df_tables) > 1:
            #numer to label different tables fromw same player
            no = 0
            for table in df_tables:
                    
                df = table.droplevel(level=0, axis=1)

                #get name
                name = webscrape_player_name(player_url)

                #to csv
                #to local file
                df.to_csv('NFL_Data_{}/{}/Players Log/{}_{}'.format(year, team, name, no))
                    
                no+=1
                    
        else:
            df = df_tables[0].droplevel(level=0, axis=1)

            #get name
            name = webscrape_player_name(player_url)

            #to csv
            df.to_csv('NFL_Data_{}/{}/Players Log/{}'.format(year, team, name))

    except:
        pass
        
    return 

In [13]:
# 2023
year=2023

In [14]:
#2022 player logs
n_a_li_2023 = []
for team in team_names:
    print(team)
    players_li = get_player_li(team=team, year=year)
    for player in players_li:    
        adv_log_player_url = final_url(player=player, year=year)
        webscrape_player_adv_log_table(player_url=adv_log_player_url, team=team, year=year)

        print(adv_log_player_url)
        
        if adv_log_player_url[:3] == 'n/a':
            n_a_li_2023.append(adv_log_player_url)

49ers
http://www.pro-football-reference.com/players/A/AiyuBr00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/A/AlleBr00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/A/ArmsAr01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BankAa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BellRo01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BosaNi00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BrenJa01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BrowJi01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BurfSp00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BurkOr00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/D/DarnSa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/D/DaviKa01/gamelog/2023/advanced/
http://www.pro-football-reference.

C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:674: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a_Robert Beal (IRD)
n/a_Samuel Womack (IRD)
n/a_Danny Gray (IR)
n/a_Drake Jackson (IR)
n/a_Cameron Latu (IR)
Bears
http://www.pro-football-reference.com/players/B/BageTy00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BaskMi00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BillAn00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BlacJo00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BlasKh00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BoroLa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/B/BrisJa01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/C/CartJa04/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/C/ColeDy00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/D/DaviNa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/D/DextGe00/gamelog/2023/advanced/
htt

C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:674: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a_T.J. Edwards
http://www.pro-football-reference.com/players/F/FeenDa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/F/FielJu00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/F/ForeDO00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/G/GateDe00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/G/GillTr01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/G/GordKy00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/G/GreeRa01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/H/HerbKh00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/H/HickEl00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/H/HomeTr00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JackEd01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JenkTe00/gamelog/2023/advanced/


...

n/a_T.J. Hockenson
http://www.pro-football-reference.com/players/H/HuntDa01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/I/IngrEd00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JackTh00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JackTr02/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JonePa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/J/JoseGr00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/L/LowrDe00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MattAl01/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MeteJo00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MullNi00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MundJo00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/M/MurpBy00/gamelog/2023/advanced/
http://www.pro-footba

C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:674: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a_K.J. Osborn
http://www.pro-football-reference.com/players/P/PaceIv00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/P/PhilHa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/P/PoweBr00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/Q/QuesDa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/R/RisnDa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/R/RoyxJa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/S/SchlAu00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/S/SmitHa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/T/ThomNa00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/T/TongKh00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/V/VigiNi00/gamelog/2023/advanced/
http://www.pro-football-reference.com/players/W/WardJa00/gamelog/2023/advanced/


C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:674: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a_Joejuan Williams
n/a_D.J. Wonnum
http://www.pro-football-reference.com/players/W/WrigRy00/gamelog/2023/advanced/
n/a_Justin Jefferson (IRD)
n/a_Chris Reed (IRD)
n/a_Cam Akers (IR)
n/a_Kirk Cousins (IR)
n/a_Marcus Davenport (IR)
n/a_Jordan Hicks (IR)
n/a_Malik Knowles (IR)
n/a_William Kwenkeu (IR)
n/a_James Lynch (IR)


In [15]:
###    N/A List   ###

n_a_li_2018 = [
    'n/a_Jordan Willis',
    'n/a_Adam Jones',
    'n/a_T.J. Carrie', 
    'n/a_J.C. Tretter',
    'n/a_Adarius Taylor', 
    'n/a_Bobo Wilson',
    'n/a_Darrel Williams',
    'n/a_Shaquille Leonard',
    'n/a_Ha Ha Clinton-Dix',
    'n/a_D.J. Swearinger',
    'n/a_T.J. McDonald',
    'n/a_B.W. Webb',
    'n/a_D.J. Hayden', 
    'n/a_James O\'Shaughnessy',
    'n/a_Robbie Chosen',
    'n/a_Don Carey', 
    'n/a_Don Muhlbach', 
    'n/a_Romeo Okwara',
    'n/a_Ha Ha Clinton-Dix', 
    'n/a_Equanimeous St. Brown',
    'n/a_Kyle Van Noy',
    'n/a_Tank Carradine', 
    'n/a_Leon Hall', 
    'n/a_Derrick Johnson',
    'n/a_Darious Williams',
    'n/a_A.J. Klein',
    'n/a_D.J. Fluker', 
    'n/a_Lano Hill',
    'n/a_Bud Dupree',
    'n/a_Buddy Howell',
    'n/a_Delanie Walker',
    'n/a_Jaleel Johnson'
]

In [16]:
n_a_li_2019 = [
     'n/a_Azeez Al-Shaair',
     'n/a_Ha Ha Clinton-Dix',
     'n/a_Sam Hubbard',
     'n/a_Kevin Huber',
     'n/a_Trenton Irwin',
     'n/a_B.W. Webb',
     'n/a_Damion Willis',
     'n/a_T.J. Carrie',
     'n/a_J.T. Hassell',
     'n/a_Adarius Taylor',
     'n/a_J.C. Tretter',
     'n/a_Sean Murphy-Bunting',
     'n/a_Bobo Wilson',
     'n/a_Jaquan Johnson',
     'n/a_D.J. Swearinger',
     'n/a_Darrel Williams',
     'n/a_Shaquille Leonard',
     'n/a_E.J. Speed',
     'n/a_Tank Carradine',
     'n/a_Jamal Davis II',
     'n/a_Jamal Perry',
     'n/a_Andrew Van Ginkel',
     'n/a_T.J. Edwards',
     'n/a_Golden Tate',
     'n/a_D.J. Hayden',
     'n/a_Charles Jones',
     "n/a_James O'Shaughnessy",
     'n/a_Brandon Rusnak',
     'n/a_Robbie Chosen',
     'n/a_Jordan Willis',
     'n/a_T.J. Hockenson',
     'n/a_C.J. Moore',
     'n/a_Greg Van Roten',
     'n/a_Michael Bennett',
     "n/a_Ja'Whaun Bentley",
     'n/a_Jakob Johnson',
     'n/a_Kyle Van Noy',
     'n/a_Joejuan Williams',
     'n/a_D.J. Swearinger',
     'n/a_Darious Williams',
     'n/a_C.J. Gardner-Johnson',
     'n/a_Deonte Harty',
     'n/a_A.J. Klein',
     'n/a_D.J. Swearinger',
     'n/a_L.J. Collier',
     'n/a_D.J. Fluker',
     'n/a_Phil Haynes',
     'n/a_Lano Hill',
     'n/a_Jamarco Jones',
     'n/a_D.K. Metcalf',
     'n/a_Bud Dupree',
     'n/a_Buddy Howell',
     'n/a_A.J. Brown',
     'n/a_Delanie Walker',
     'n/a_Jaleel Johnson'
]

In [17]:
n_a_li_2020 = [
     'n/a_Azeez Al-Shaair',
     'n/a_Darrion Daniels',
     'n/a_Ross Dwelley',
     'n/a_Jordan Willis',
     'n/a_Jaylon Johnson',
     'n/a_Sherrick McManis',
     'n/a_Lamar Miller',
     'n/a_Jonah Williams',
     'n/a_P.J. Locke',
     'n/a_J.C. Tretter',
     'n/a_Sean Murphy-Bunting',
     'n/a_A.J. Epenesa',
     'n/a_Jaquan Johnson',
     'n/a_A.J. Klein',
     'n/a_K.J. Hill',
     'n/a_Tyree St. Louis',
     'n/a_Willie Gay Jr.',
     'n/a_Darrel Williams',
     'n/a_T.J. Carrie',
     'n/a_DeMichael Harris',
     'n/a_Shaquille Leonard',
     'n/a_E.J. Speed',
     'n/a_Chase Roullier',
     'n/a_Tim Settle',
     'n/a_James Smith-Williams',
     'n/a_Hunter Niswander',
     'n/a_Jamal Perry',
     'n/a_Andrew Van Ginkel',
     'n/a_Kyle Van Noy',
     'n/a_T.J. Edwards',
     'n/a_Sua Opeta',
     'n/a_Prince Tega Wanogho',
     'n/a_T.J. Brunson',
     'n/a_Jarren Williams',
     'n/a_D.J. Hayden',
     'n/a_C.J. Henderson',
     "n/a_James O'Shaughnessy",
     'n/a_Brandon Rusnak',
     'n/a_J.T. Hassell',
     'n/a_Lamar Jackson',
     'n/a_Jordan Jenkins',
     'n/a_Greg Van Roten',
     'n/a_Jordan Willis',
     'n/a_T.J. Hockenson',
     'n/a_C.J. Moore',
     "n/a_De'Jon Harris",
     'n/a_Equanimeous St. Brown',
     'n/a_Robbie Chosen',
     'n/a_Adarius Taylor',
     'n/a_P.J. Walker',
     'n/a_Jakob Johnson',
     'n/a_J.J. Taylor',
     'n/a_Joejuan Williams',
     'n/a_Jaryd Jones-Smith',
     'n/a_Darious Williams',
     'n/a_J.K. Dobbins',
     'n/a_D.J. Fluker',
     'n/a_C.J. Gardner-Johnson',
     'n/a_Deonte Harty',
     'n/a_D.J. Swearinger',
     'n/a_L.J. Collier',
     'n/a_Lano Hill',
     'n/a_Jamarco Jones',
     'n/a_D.K. Metcalf',
     'n/a_Bud Dupree',
     'n/a_Buddy Howell',
     'n/a_A.J. Brown',
     'n/a_Jaleel Johnson',
     'n/a_K.J. Osborn',
     'n/a_D.J. Wonnum'
]

In [18]:
n_a_li_2021 = [
    'n/a_Azeez Al-Shaair',
     'n/a_Jordan Willis',
     'n/a_Jaylon Johnson',
     'n/a_Khyiris Tonga',
     'n/a_Danny Trevathan',
     'n/a_Trey Hopkins',
     'n/a_Jonah Williams',
     'n/a_Jamar Johnson',
     'n/a_P.J. Locke',
     'n/a_Javonte Williams',
     'n/a_J.C. Tretter',
     'n/a_K.J. Britt',
     'n/a_Sean Murphy-Bunting',
     'n/a_Boogie Basham',
     'n/a_A.J. Epenesa',
     'n/a_Jaquan Johnson',
     'n/a_A.J. Klein',
     'n/a_K.J. Hill',
     'n/a_Willie Gay Jr.',
     'n/a_Prince Tega Wanogho',
     'n/a_Darrel Williams',
     'n/a_T.J. Carrie',
     'n/a_DeMichael Harris',
     'n/a_Shaquille Leonard',
     'n/a_E.J. Speed',
     'n/a_Jamin Davis',
     "n/a_De'Jon Harris",
     'n/a_D.J. Hayden',
     'n/a_James Smith-Williams',
     'n/a_Benjamin St-Juste',
     'n/a_Tarell Basham',
     'n/a_Amari Cooper',
     'n/a_Javaris Davis',
     'n/a_Jamal Perry',
     'n/a_Andrew Van Ginkel',
     'n/a_T.J. Edwards',
     'n/a_Sua Opeta',
     'n/a_Milton Williams',
     'n/a_Jarren Williams',
     'n/a_C.J. Henderson',
     'n/a_Jamir Jones',
     "n/a_James O'Shaughnessy",
     'n/a_Brandon Rusnak',
     'n/a_D.J. Montgomery',
     'n/a_Greg Van Roten',
     'n/a_Jack Fox',
     'n/a_Mark Gilbert',
     'n/a_Jared Goff',
     'n/a_T.J. Hockenson',
     'n/a_C.J. Moore',
     'n/a_Amon-Ra St. Brown',
     'n/a_Equanimeous St. Brown',
     'n/a_Robbie Chosen',
     'n/a_C.J. Henderson',
     'n/a_C.J. Saunders',
     'n/a_P.J. Walker',
     'n/a_Jakob Johnson',
     'n/a_J.J. Taylor',
     'n/a_Kyle Van Noy',
     'n/a_Joejuan Williams',
     'n/a_Ha Ha Clinton-Dix',
     'n/a_Buddy Howell',
     'n/a_Jamir Jones',
     'n/a_J.J. Koski',
     'n/a_Darious Williams',
     'n/a_Jonah Williams',
     'n/a_Jaryd Jones-Smith',
     'n/a_Odafe Oweh',
     'n/a_C.J. Gardner-Johnson',
     'n/a_Deonte Harty',
     'n/a_L.J. Collier',
     'n/a_Jamarco Jones',
     'n/a_D.K. Metcalf',
     'n/a_Jamir Jones',
     'n/a_Jaleel Johnson',
     'n/a_A.J. Brown',
     'n/a_Bud Dupree',
     'n/a_K.J. Osborn',
     'n/a_T.J. Smith',
     'n/a_D.J. Wonnum'
              ]

In [19]:
n_a_li_2022 = [
    'n/a_Azeez Al-Shaair',
     'n/a_Jordan Willis',
     'n/a_Jaylon Johnson',
     'n/a_Braxton Jones',
     'n/a_Jaylon Jones',
     'n/a_A.J. Klein',
     'n/a_Michael Schofield',
     'n/a_Equanimeous St. Brown',
     'n/a_A.J. Thomas',
     'n/a_Jonah Williams',
     'n/a_P.J. Locke',
     'n/a_Javonte Williams',
     'n/a_Jordan Elliott',
     'n/a_Martin Emerson',
     'n/a_K.J. Britt',
     'n/a_Sean Murphy-Bunting',
     'n/a_J.J. Russell',
     'n/a_Boogie Basham',
     'n/a_C.J. Brewer',
     'n/a_A.J. Epenesa',
     'n/a_Jaquan Johnson',
     'n/a_A.J. Klein',
     'n/a_Greg Van Roten',
     'n/a_Robbie Chosen',
     'n/a_Darrel Williams',
     'n/a_Kyle Van Noy',
     'n/a_Willie Gay Jr.',
     'n/a_Prince Tega Wanogho',
     'n/a_Shaquille Leonard',
     'n/a_E.J. Speed',
     'n/a_Jamin Davis',
     "n/a_De'Jon Harris",
     'n/a_James Smith-Williams',
     'n/a_Benjamin St-Juste',
     'n/a_Andrew Van Ginkel',
     'n/a_A.J. Brown',
     'n/a_T.J. Edwards',
     'n/a_C.J. Gardner-Johnson',
     'n/a_Sua Opeta',
     'n/a_Milton Williams',
     'n/a_Jaleel Johnson',
     'n/a_D.J. Davidson',
     'n/a_Darious Williams',
     'n/a_T.J. Hockenson',
     'n/a_C.J. Moore',
     'n/a_Amon-Ra St. Brown',
     'n/a_Jameson Williams',
     'n/a_Jarren Williams',
     'n/a_T.J. Carrie',
     'n/a_Robbie Chosen',
     'n/a_C.J. Henderson',
     'n/a_P.J. Walker',
     'n/a_Jack Jones',
     'n/a_Calvin Munson',
     'n/a_Bill Murray',
     'n/a_Michael Onwenu',
     'n/a_J.J. Taylor',
     'n/a_Raleigh Webb',
     'n/a_Deatrich Wise Jr.',
     'n/a_Curtis Bolton',
     'n/a_Jakob Johnson',
     'n/a_T.J. Carter',
     'n/a_T.J. Carter',
     'n/a_Cobie Durant',
     'n/a_Jonah Williams',
     'n/a_J.K. Dobbins',
     'n/a_A.J. Klein',
     'n/a_Odafe Oweh',
     'n/a_Damarion Williams',
     'n/a_Deonte Harty',
     'n/a_L.J. Collier',
     'n/a_D.K. Metcalf',
     'n/a_Riq Woolen',
     'n/a_Jamir Jones',
     'n/a_Jaleel Johnson',
     'n/a_Bud Dupree',
     'n/a_Malik Willis',
     'n/a_T.J. Hockenson',
     'n/a_K.J. Osborn',
     'n/a_T.J. Smith',
     'n/a_D.J. Wonnum'
]

In [21]:
n_a_li_2023 = [
     'n/a_Robert Beal (IRD)',
     'n/a_Samuel Womack (IRD)',
     'n/a_Danny Gray (IR)',
     'n/a_Drake Jackson (IR)',
     'n/a_Cameron Latu (IR)',
     'n/a_T.J. Edwards',
     'n/a_Jaylon Johnson',
     'n/a_Braxton Jones',
     'n/a_Jaylon Jones',
     'n/a_Equanimeous St. Brown',
     'n/a_Jake Browning',
     'n/a_Jonah Williams',
     'n/a_Tycen Anderson (IR)',
     'n/a_Chase Brown (IR)',
     'n/a_Devin Cochran (IR)',
     'n/a_P.J. Locke',
     'n/a_Chris Manhertz',
     'n/a_Javonte Williams',
     'n/a_Eyioma Uwazurike (SUS)',
     'n/a_Greg Dulcich (IR)',
     'n/a_Jonas Griffith (IR)',
     'n/a_Brandon Johnson (IR)',
     'n/a_Alex Palczewski (IR)',
     'n/a_Tim Patrick (IR)',
     'n/a_Caden Sterns (IR)',
     'n/a_Jalen Virgil (IR)',
     'n/a_P.J. Walker',
     'n/a_Drew Forbes (NON)',
     'n/a_Michael Woods II (NON)',
     'n/a_Daylen Baldwin (IR)',
     'n/a_Nick Chubb (IR)',
     'n/a_Jack Conklin (IR)',
     'n/a_Dawson Deaton (IR)',
     'n/a_Thomas Graham (IR)',
     'n/a_Jakeem Grant (IR)',
     'n/a_Jordan Kunaszyk (IR)',
     'n/a_Cameron Mitchell (IR)',
     'n/a_Ty Nsekhe (IR)',
     'n/a_Jacob Phillips (IR)',
     'n/a_Deshaun Watson (IR)',
     'n/a_K.J. Britt',
     'n/a_Silas Dzansi (IR)',
     'n/a_Russell Gage (IR)',
     'n/a_A.J. Epenesa',
     'n/a_Deonte Harty',
     'n/a_A.J. Klein',
     'n/a_Ryan Van Demark',
     'n/a_Nyheim Hines (NON)',
     'n/a_Kameron Cline (IR)',
     'n/a_Zach Davidson (IR)',
     'n/a_Tommy Doyle (IR)',
     'n/a_Kaiir Elam (IR)',
     'n/a_Damien Harris (IR)',
     'n/a_DaQuan Jones (IR)',
     'n/a_Dawson Knox (IR)',
     'n/a_Matt Milano (IR)',
     'n/a_Justin Shorter (IR)',
     'n/a_L.J. Collier (IR)',
     'n/a_Pat Elflein (IR)',
     'n/a_Zach Ertz (IR)',
     'n/a_Rashad Fenton (IR)',
     'n/a_Jon Gaines (IR)',
     'n/a_Matt Hembrough (IR)',
     'n/a_Hayden Howerton (IR)',
     'n/a_Marlon Mack (IR)',
     'n/a_Bobby Price (IR)',
     'n/a_Carlos Watkins (IR)',
     'n/a_Corey Linsley (NON)',
     'n/a_JT Woods (IRD)',
     'n/a_Raheem Layne (IR)',
     'n/a_Josh Palmer (IR)',
     'n/a_Chris Rumph (IR)',
     'n/a_Andrew Trainer (IR)',
     'n/a_Amechi Uzodinma (IR)',
     'n/a_Willie Gay Jr.',
     'n/a_Nick Bolton (IR)',
     'n/a_Jody Fortson (IR)',
     'n/a_Nazeeh Johnson (IR)',
     'n/a_Prince Tega Wanogho (IR)',
     'n/a_Jaylon Jones',
     'n/a_Shaquille Leonard',
     'n/a_E.J. Speed',
     'n/a_Grover Stewart (SUS)',
     'n/a_Genard Avery (IR)',
     'n/a_Ashton Dulin (IR)',
     'n/a_Dallis Flowers (IR)',
     'n/a_Evan Hull (IR)',
     'n/a_Titus Leo (IR)',
     'n/a_Danny Pinter (IR)',
     'n/a_Anthony Richardson (IR)',
     'n/a_Daniel Scott (IR)',
     'n/a_Isaac Taylor-Stuart (IR)',
     'n/a_Jake Witt (IR)',
     'n/a_Jamin Davis',
     "n/a_De'Jon Harris",
     'n/a_K.J. Henry',
     'n/a_Jartavius Martin',
     'n/a_James Smith-Williams',
     'n/a_Benjamin St-Juste',
     'n/a_Shaka Toney (SUS)',
     'n/a_Troy Apke (IR)',
     'n/a_David Bada (IR)',
     'n/a_Cody Barton (IR)',
     'n/a_Curtis Brooks (IR)',
     'n/a_Saahdiq Charles (IR)',
     'n/a_Braeden Daniels (IR)',
     'n/a_Darrick Forrest (IR)',
     'n/a_Dax Milne (IR)',
     'n/a_Jeremy Reaves (IR)',
     'n/a_Armani Rogers (IR)',
     'n/a_T.J. Bass',
     'n/a_Peyton Hendershot (IRD)',
     'n/a_Josh Ball (IR)',
     'n/a_Trevon Diggs (IR)',
     'n/a_David Durden (IR)',
     'n/a_Viliami Fehoko (IR)',
     'n/a_C.J. Goodwin (IR)',
     'n/a_DeMarvion Overshown (IR)',
     'n/a_Billy Price (IR)',
     'n/a_John Stephens (IR)',
     'n/a_Leighton Vander Esch (IR)',
     'n/a_Robbie Chosen',
     'n/a_Andrew Van Ginkel',
     'n/a_Erik Ezukanma (NON)',
     'n/a_Chris Brooks (IR)',
     'n/a_Keion Crossen (IR)',
     'n/a_Zeke Vandenburgh (IR)',
     'n/a_A.J. Brown',
     'n/a_Sua Opeta',
     'n/a_Milton Williams',
     'n/a_Isaiah Rodgers (SUS)',
     'n/a_Justin Evans (IRD)',
     'n/a_Shaun Bradley (IR)',
     'n/a_Nakobe Dean (IR)',
     'n/a_Roderick Johnson (IR)',
     'n/a_Avonte Maddox (IR)',
     'n/a_Zech McPhearson (IR)',
     'n/a_Bud Dupree',
     'n/a_Josh Ali (IR)',
     'n/a_Troy Andersen (IR)',
     'n/a_Ikenna Enechukwu (IR)',
     'n/a_Feleipe Franks (IR)',
     'n/a_Ethan Greenidge (IR)',
     'n/a_Matt Hennessy (IR)',
     'n/a_Parker Hesse (IR)',
     'n/a_Grady Jarrett (IR)',
     'n/a_LaCale London (IR)',
     'n/a_Adetokunbo Ogundeji (IR)',
     'n/a_Barry Wesley (IR)',
     'n/a_Avery Williams (IR)',
     'n/a_Boogie Basham',
     'n/a_D.J. Davidson',
     'n/a_Aaron Robinson (PUP)',
     'n/a_Tommy Sweeney (NON)',
     'n/a_Gary Brightwell (IR)',
     'n/a_Jarrad Davis (IR)',
     'n/a_Joshua Ezeudu (IR)',
     'n/a_Bryce Ford-Wheaton (IR)',
     'n/a_Graham Gano (IR)',
     'n/a_Eric Gray (IR)',
     'n/a_J.C. Hassenauer (IR)',
     'n/a_Daniel Jones (IR)',
     'n/a_Shane Lemieux (IR)',
     'n/a_Matt Peart (IR)',
     'n/a_Tyrod Taylor (IR)',
     'n/a_Darious Williams',
     'n/a_Jamal Agnew (IR)',
     'n/a_Christian Braswell (IR)',
     'n/a_Cooper Hodges (IR)',
     'n/a_Ventrell Miller (IR)',
     'n/a_Jaylon Moore (IR)',
     'n/a_Duane Brown (IRD)',
     'n/a_Kenny Yeboah (IRD)',
     'n/a_Bradlee Anae (IR)',
     'n/a_Chuck Clark (IR)',
     'n/a_Javelin Guidry (IR)',
     'n/a_Maalik Hall (IR)',
     'n/a_Justin Hardee (IR)',
     'n/a_Connor McGovern (IR)',
     'n/a_Ifeadi Odenigbo (IR)',
     'n/a_Aaron Rodgers (IR)',
     'n/a_Wes Schweitzer (IR)',
     'n/a_Alijah Vera-Tucker (IR)',
     'n/a_Amon-Ra St. Brown',
     'n/a_Jameson Williams',
     'n/a_Hendon Hooker (NON)',
     'n/a_Jason Cabinda (IR)',
     'n/a_Scott Daly (IR)',
     'n/a_C.J. Gardner-Johnson (IR)',
     'n/a_James Houston (IR)',
     'n/a_Mohamed Ibrahim (IR)',
     'n/a_Zonovan Knight (IR)',
     'n/a_Emmanuel Moseley (IR)',
     'n/a_Matt Nelson (IR)',
     'n/a_Nate Sudfeld (IR)',
     'n/a_Halapoulivaati Vaitai (IR)',
     'n/a_Lukas Van Ness',
     'n/a_David Bakhtiari (IR)',
     'n/a_Tyler Davis (IR)',
     'n/a_Darnell Savage Jr. (IR)',
     'n/a_Eric Stokes (IR)',
     'n/a_C.J. Henderson',
     'n/a_Jaycee Horn (IRD)',
     'n/a_Henry Anderson (IR)',
     'n/a_Claudin Cherelus (IR)',
     'n/a_Jeremy Chinn (IR)',
     'n/a_Brady Christensen (IR)',
     'n/a_Tarik Cohen (IR)',
     'n/a_Yetur Gross-Matos (IR)',
     'n/a_Justin Houston (IR)',
     'n/a_Giovanni Ricci (IR)',
     'n/a_Shaq Thompson (IR)',
     'n/a_Luiji Vilain (IR)',
     'n/a_Riley Reiff (IRD)',
     'n/a_Calvin Anderson (IR)',
     'n/a_Isaiah Bolden (IR)',
     'n/a_Kendrick Bourne (IR)',
     'n/a_Daniel Ekuale (IR)',
     'n/a_Christian Gonzalez (IR)',
     'n/a_Marcus Jones (IR)',
     'n/a_Matt Judon (IR)',
     'n/a_Raekwon McMillan (IR)',
     'n/a_Tre Nixon (IR)',
     'n/a_Jack Jones',
     'n/a_Christopher Smith',
     'n/a_Jaylon Smith',
     'n/a_Greg Van Roten',
     'n/a_Roderic Teamer (IRD)',
     'n/a_Brittain Brown (IR)',
     'n/a_Darien Butler (IR)',
     'n/a_Brandon Facyson (IR)',
     "n/a_Kana'i Mauga (IR)",
     'n/a_Dalton Wagner (IR)',
     'n/a_Cobie Durant',
     'n/a_Desjuan Johnson',
     "n/a_Tre'Vius Tomlinson",
     'n/a_Jonah Williams',
     'n/a_Stetson Bennett (NON)',
     'n/a_Ronnie Rivers (IR)',
     'n/a_Jason Taylor (IR)',
     'n/a_Odafe Oweh',
     'n/a_Kyle Van Noy',
     'n/a_Tyus Bowser (NON)',
     'n/a_Nick Moore (NON)',
     'n/a_Trayvon Mullen (NON)',
     'n/a_Andrew Vorhees (NON)',
     'n/a_J.K. Dobbins (IR)',
     'n/a_Malik Hamm (IR)',
     'n/a_David Ojabo (IR)',
     "n/a_Ar'Darius Washington (IR)",
     'n/a_A.T. Perry',
     'n/a_Eno Benjamin (IR)',
     'n/a_Ryan Connelly (IR)',
     'n/a_Andrew Dowell (IR)',
     'n/a_Payton Turner (IR)',
     'n/a_D.K. Metcalf',
     'n/a_Riq Woolen',
     'n/a_Bryan Mone (PUP)',
     'n/a_Abraham Lucas (IRD)',
     'n/a_Coby Bryant (IR)',
     'n/a_Mike Morris (IR)',
     'n/a_Uchenna Nwosu (IR)',
     'n/a_Josh Onujiogu (IR)',
     'n/a_Broderick Jones',
     'n/a_Kwon Alexander (IR)',
     'n/a_Cole Holcomb (IR)',
     'n/a_Keanu Neal (IR)',
     'n/a_Cory Trice (IR)',
     'n/a_Tank Dell',
     'n/a_C.J. Stroud',
     'n/a_Denzel Perryman (SUS)',
     'n/a_Charlie Heck (IRD)',
     'n/a_Grayland Arnold (IR)',
     "n/a_Ka'imi Fairbairn (IR)",
     'n/a_Kendrick Green (IR)',
     'n/a_Kenyon Green (IR)',
     'n/a_Troy Hairston (IR)',
     'n/a_Jesse Matthews (IR)',
     'n/a_Eric Murray (IR)',
     'n/a_Jarrett Patterson (IR)',
     'n/a_Scott Quessenberry (IR)',
     'n/a_Teagan Quitoriano (IR)',
     'n/a_Hassan Ridgeway (IR)',
     'n/a_DJ Scaife (IR)',
     'n/a_Juice Scruggs (IR)',
     'n/a_M.J. Stewart (IR)',
     'n/a_Azeez Al-Shaair',
     'n/a_Jaleel Johnson',
     'n/a_Sean Murphy-Bunting',
     'n/a_Malik Willis',
     'n/a_Caleb Farley (PUP)',
     'n/a_Shakel Brown (IR)',
     'n/a_Chance Campbell (IR)',
     'n/a_Julius Chestnut (IR)',
     'n/a_Hassan Haskins (IR)',
     'n/a_Kearis Jackson (IR)',
     'n/a_T.J. Hockenson',
     'n/a_K.J. Osborn',
     'n/a_Joejuan Williams',
     'n/a_D.J. Wonnum',
     'n/a_Justin Jefferson (IRD)',
     'n/a_Chris Reed (IRD)',
     'n/a_Cam Akers (IR)',
     'n/a_Kirk Cousins (IR)',
     'n/a_Marcus Davenport (IR)',
     'n/a_Jordan Hicks (IR)',
     'n/a_Malik Knowles (IR)',
     'n/a_William Kwenkeu (IR)',
     'n/a_James Lynch (IR)'
]